<a href="https://colab.research.google.com/github/Sushmithaa-Pandian/Machine_learning_sush/blob/main/Real_combine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch


In [ ]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 12.6 MB/s eta 0:00:00


In [ ]:
from google.colab import files
uploaded=files.upload()

Saving Real_Combine.csv to Real_Combine.csv


In [ ]:
df=pd.read_csv("Real_Combine.csv")

In [ ]:
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [ ]:
x=df.iloc[:,:-1]
y=df.iloc[:,-1]


In [ ]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
    model.add(layers.Dense(1, activation='linear'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mean_absolute_error',
        metrics=['mean_absolute_error'])
    return model


In [ ]:
tuner = RandonSearch(
    build_model,
    objective='val_mean_absolute_error',
    max_trials=5,
    executions_per_trial=3,
    directory='project',
    project_name='Air Quality Index')

NameError: ignored

In [ ]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)

In [ ]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(x_train, y_train, epochs=50, validation_split=0.2)



Epoch 1/50
20/20 [==============================] - 5s 38ms/step - loss: 98.7130 - mean_absolute_error: 98.7130 - val_loss: nan - val_mean_absolute_error: nan
Epoch 2/50
20/20 [==============================] - 0s 17ms/step - loss: 73.6142 - mean_absolute_error: 73.6142 - val_loss: nan - val_mean_absolute_error: nan
Epoch 3/50
20/20 [==============================] - 0s 17ms/step - loss: 70.3773 - mean_absolute_error: 70.3773 - val_loss: nan - val_mean_absolute_error: nan
Epoch 4/50
20/20 [==============================] - 0s 11ms/step - loss: 63.2106 - mean_absolute_error: 63.2106 - val_loss: nan - val_mean_absolute_error: nan
Epoch 5/50
20/20 [==============================] - 0s 10ms/step - loss: 61.4532 - mean_absolute_error: 61.4532 - val_loss: nan - val_mean_absolute_error: nan
Epoch 6/50
20/20 [==============================] - 0s 11ms/step - loss: 54.2509 - mean_absolute_error: 54.2509 - val_loss: nan - val_mean_absolute_error: nan
Epoch 7/50
20/20 [============================

In [ ]:
eval_result = model.evaluate(x_test, y_test)
print(eval_result)

11/11 [==============================] - 0s 3ms/step - loss: 44.7552 - mean_absolute_error: 44.7552
[44.75521469116211, 44.75521469116211]
